# Modules run after SVM predictions.

## Includes merging of alternate count information, filtering based on SNPKebab predictions, and filtering of samples based on median depth

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import re
import matplotlib.pyplot as plt

In [6]:
#aggregate mafs that have annotation information that is missing from phylogic outputs
import shutil
import glob

outfilename='trimmed_mafs/allMaf_trim.txt'

with open(outfilename, 'wb') as outfile:
    for filename in glob.glob('trimmed_mafs/*.aggregated.maf'):
        if filename == outfilename:
            # don't want to copy the output into the output
            continue
        with open(filename, 'rb') as readfile:
            shutil.copyfileobj(readfile, outfile)

In [3]:
anno_file="~/Documents/COPD/copd_minimal_annotated.txt"
phylo_file="~/Documents/COPD/all_clone/noLod/COPD_noLod_noSegdup_hwe_1kg_CNV_map.txt"

anno_maf=pd.read_csv(anno_file,sep='\t',dtype='object')
phylo_maf=pd.read_csv(phylo_file,sep='\t',dtype='object')
#pd.DataFrame(anno_maf.columns).to_csv("maf_columns.txt",index=False)

#identify annotations to append to the phylogic MAFs and include annotations that allow for complete merging
anno_maf=anno_maf.loc[:,["Hugo_Symbol","Chromosome","Start_position",
                     "End_position","Reference_Allele","Tumor_Seq_Allele2",'Tumor_Sample_Barcode',"t_alt_count",
                        "t_ref_count","SwissProt_acc_Id","SwissProt_entry_Id","ref_context","pon_loglike",
                        "i_t_ALT_F1R2","i_t_ALT_F2R1","i_t_REF_F1R2","i_t_REF_F2R1","t_lod_fstar"]]

print(len(phylo_maf.index))
print(len(phylo_maf['Tumor_Sample_Barcode'].unique()))

merge_maf=phylo_maf.merge(anno_maf,on=["Hugo_Symbol","Chromosome","Start_position",
                     "End_position","Tumor_Sample_Barcode","Reference_Allele","Tumor_Seq_Allele2"])

print(len(merge_maf['Tumor_Sample_Barcode'].unique()))

merge_maf.to_csv("~/Documents/COPD/all_clone/noLod/reannotated/SVM_predicted_phylogic_muts_counts_all.txt",sep='\t',index=False)

print(len(merge_maf.index))


58909
2541
2541
58909


In [10]:
#filter mutations based on SVM predictions
input_file="~/Documents/COPD/all_clone/noLod/reannotated/SVM_pred_phylogic_muts_reheaded.txt"

maf=pd.read_csv(input_file,sep='\t')

som_lik=stats.beta.pdf(maf['CCF_hat_01'],maf['t_alt_count']+1,
                       maf['t_ref_count']+1)

germ_lik=stats.beta.pdf(maf['predicted_germ_af'],maf['t_alt_count']+1,
                       maf['t_ref_count']+1)

ratio=som_lik/(som_lik+germ_lik)

maf=maf[ratio>0.9]

#filtered_count=maf.groupby(['Tumor_Sample_Barcode']).size().reset_index(name='counts')

#count_chip=filtered_count.loc[filtered_count.counts>=4,:]


maf.to_csv("~/Documents/COPD/all_clone/noLod/reannotated/SVM_pred_phylogic_muts_filt.txt",sep='\t',index=False)

Index(['Hugo_Symbol', 'Entrez_Gene_Id', 'Center', 'NCBI_Build', 'Chromosome',
       'Start_position', 'End_position', 'Strand', 'Variant_Classification',
       'Variant_Type', 'Reference_Allele', 'Tumor_Seq_Allele1',
       'Tumor_Seq_Allele2', 'Protein_Change', 'Cluster_ID', 'CCF_hat_01',
       'CCF90_low_01', 'CCF90_high_01', 'Tumor_Sample_Barcode',
       'germline_cluster', 'hwe_fail', 'gain_overlap', 'has_segdup',
       't_alt_count', 't_ref_count', 'SwissProt_acc_Id', 'SwissProt_entry_Id',
       'ref_context', 'pon_loglike', 'i_t_ALT_F1R2', 'i_t_ALT_F2R1',
       'i_t_REF_F1R2', 'i_t_REF_F2R1', 't_lod_fstar', 'Chromosome_num',
       'distance_to_bait', 'gc_content', 'score', 'numbered_Chromosome',
       'genomic_position', 'score.1', 'numbered_Chromosome_y', 'tumor_f',
       'ref_allele', 'context', 'contig', 'position', 'hwe_fail_y',
       'has_segdup_y', 'gain_overlap_y', 'loss_overlap', 'score_y', 'af_next',
       'af_prev', 'less_than_0.1', 'between_0.1_0.2', 'betwe

In [2]:
### fix ID names ###

anno_file="whole_set_valid/exac_functional.blat.maf"
anno_maf=pd.read_csv(anno_file,sep='\t',dtype='object')

key_file="exac_full_set_hyphenID_key.txt"
key=pd.read_csv(key_file,sep='\t',dtype='object')

anno_maf=anno_maf.drop_duplicates(keep='first')
anno_maf=anno_maf.reset_index(drop=True)
print([len(anno_maf['Tumor_Sample_Barcode'].unique()),len(anno_maf.index)])

merge_maf=anno_maf.merge(key,left_on='Tumor_Sample_Barcode',right_on='key',how='left')

merge_maf=merge_maf.drop_duplicates(keep='first')
merge_maf=merge_maf.reset_index(drop=True)

merge_maf.loc[
    ~merge_maf['sample_id'].isnull(),'Tumor_Sample_Barcode']=merge_maf.loc[
    ~merge_maf['sample_id'].isnull(),'sample_id']

print([len(merge_maf['Tumor_Sample_Barcode'].unique()),len(merge_maf.index)])

merge_maf.to_csv("whole_set_valid/validation_snv_revised_sample_names.txt",sep='\t',index=False)




[7087, 8982]
[7089, 8984]


Generate false discovery rate per sample based on median depth 

In [39]:
# purity_range=np.arange(0.01,0.51,by=0.01)
# depth_range=np.arange(10,410,by=10)

plt.gcf().clear()
iterations=10000

in_file="all_clone/all_clone_mut_lim/top_clone_ageMutLeyLim.txt"
maf=pd.read_csv(in_file,sep='\t')

maf_subset=maf.loc[:,["Tumor_Sample_Barcode","t_alt_count","t_ref_count","CCF_hat_01","germline_cluster"]]

print(len(maf_subset["Tumor_Sample_Barcode"].unique()))

maf_subset['depth']=maf_subset.t_ref_count+maf_subset.t_alt_count

median_depths=maf_subset.groupby(['Tumor_Sample_Barcode'])['depth'].median().reset_index(name='median_depth')

maf_subset=pd.merge(maf_subset,median_depths,on="Tumor_Sample_Barcode")
maf_subset=maf_subset.drop_duplicates(subset=["Tumor_Sample_Barcode"])
maf_subset['median_depth']=maf_subset['median_depth'].astype(int)
maf_subset=maf_subset.reset_index(drop=True)

confusion_df=pd.DataFrame(columns=["Tumor_Sample_Barcode","FDR"])

for idx,row in maf_subset.iterrows():

    somatic_sim_alt=stats.binom.rvs(row['median_depth'],row['CCF_hat_01'],size=iterations)
    germ_sim_alt=stats.binom.rvs(row['median_depth'],row['germline_cluster'],size=iterations)
    
    sim_germ_germ=stats.beta.pdf(row['germline_cluster'],germ_sim_alt+1,row['median_depth']-germ_sim_alt+1)
    sim_germ_somatic=stats.beta.pdf(row['germline_cluster'],somatic_sim_alt+1,row['median_depth']-somatic_sim_alt+1)
    
    sim_somatic_germ=stats.beta.pdf(row['CCF_hat_01'],germ_sim_alt+1,row['median_depth']-germ_sim_alt+1)
    sim_somatic_somatic=stats.beta.pdf(row['CCF_hat_01'],somatic_sim_alt+1,row['median_depth']-somatic_sim_alt+1)
    
    som_lod=np.subtract(np.log10(sim_somatic_somatic),np.log10(sim_germ_somatic))
    germ_lod=np.subtract(np.log10(sim_somatic_germ),np.log10(sim_germ_germ))
    
    min_lik=min(min(som_lod),min(germ_lod))
    max_lik=max(max(som_lod),max(germ_lod))
    #print([min_lik,max_lik])
#     min_lik=-10
#     max_lik=10
    
    threshold=np.arange(min_lik,max_lik,abs(min_lik-max_lik)/99)
    
    true_positive=[]
    false_negative=[]
    true_negative=[]
    false_positive=[]

    for theta in threshold:

        true_positive.append(len(som_lod[som_lod>=theta]))
        false_negative.append(len(som_lod[(som_lod<theta)|(somatic_sim_alt<=3)]))
        true_negative.append(len(germ_lod[(germ_lod<theta)|(germ_sim_alt<=3)]))
        false_positive.append(len(germ_lod[germ_lod>=theta]))

    tp=np.array(true_positive)/(np.array(true_positive)+np.array(false_negative))
    fp=np.array(false_positive)/(np.array(false_positive)+np.array(true_negative))
    fd=np.array(false_positive)/(np.array(false_positive)+(1/1000)*np.array(true_positive))
    
#     if idx<10:
#         plt.plot(fd,tp)
#         plt.xlabel("FDR")
#         plt.ylabel("TPR")
#         #plt.legend(loc='left')
#     else:
#         plt.savefig("fdr_by_purity_and_depth.png")
#         break
    

    if len(np.where(tp>=0.9)[0])>0:
        print([row['Tumor_Sample_Barcode'],min(fp[tp>=0.9]),row['median_depth'],row['CCF_hat_01'],
              row['germline_cluster']])
        
        df=pd.DataFrame([[row['Tumor_Sample_Barcode'],min(fd[tp>=0.9])]],
                                 index=[0],columns=["Tumor_Sample_Barcode","FDR"])
        
        confusion_df=confusion_df.append(df,ignore_index=True)
    else:
        df=pd.DataFrame([[row['Tumor_Sample_Barcode'],0]],
                       index=[0],columns=["Tumor_Sample_Barcode","FDR"])
        confusion_df=confusion_df.append(df,ignore_index=True)


#confusion_df.to_csv('confusion.txt',sep='\t',index=False)

confusion_df=confusion_df.loc[confusion_df['FDR']<0.1,:]

maf=maf[maf["Tumor_Sample_Barcode"].isin(confusion_df["Tumor_Sample_Barcode"])]

maf.to_csv("all_clone/all_clone_mut_lim/depth_lim_ageMutLeyLim.txt",sep='\t',index=False)
    
#plt.savefig("fdr_by_purity_and_depth.png")
    
    #print(confusion_df)
    

1890
['0025B', 0.0917, 98, 0.35, 0.48]
['0032A', 0.0716, 106, 0.34, 0.47]
['0041A', 0.1392, 91, 0.35, 0.48]
['0054B', 0.0834, 94, 0.34, 0.48]
['0060B', 0.0, 143, 0.06, 0.47]
['0066A', 0.1386, 80, 0.35, 0.48]
['0067A', 0.0851, 110, 0.34, 0.47]
['0099A', 0.1877, 59, 0.35, 0.49]
['01-0469', 0.0039, 189, 0.35, 0.49]
['0111B', 0.2, 50, 0.32, 0.47]
['0135A', 0.1414, 53, 0.31, 0.48]
['0138A', 0.1042, 86, 0.34, 0.48]
['0206B', 0.3037, 29, 0.29, 0.48]
['0251A', 0.079, 117, 0.35, 0.48]
['0252A', 0.0511, 106, 0.33, 0.47]
['0268A', 0.1074, 67, 0.31, 0.47]
['0269A', 0.0, 147, 0.05, 0.54]
['0310A', 0.1136, 97, 0.35, 0.48]
['0322B', 0.0605, 75, 0.3, 0.47]
['0324B', 0.1076, 102, 0.34, 0.47]
['0332A', 0.1761, 78, 0.34, 0.47]
['0347A', 0.1266, 64, 0.31, 0.47]
['0354B', 0.0, 120, 0.07, 0.47]


/Users/mleventh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in true_divide


['0359B', 0.1989, 63, 0.35, 0.49]
['0372A', 0.0, 192, 0.04, 0.46]
['0375A', 0.0, 109, 0.06, 0.47]
['0385B', 0.1241, 80, 0.33, 0.47]
['0398A', 0.0, 73, 0.09, 0.45]
['04-0313', 0.0783, 34, 0.26, 0.49]
['0402A', 0.0296, 153, 0.34, 0.47]
['0496A', 0.1921, 93, 0.35, 0.47]
['0501A', 0.0118, 220, 0.34, 0.46]
['0516A', 0.1954, 66, 0.34, 0.47]
['0535B', 0.0571, 103, 0.35, 0.49]
['0551A', 0.1778, 62, 0.32, 0.47]
['0567B', 0.1386, 80, 0.34, 0.48]
['0588B', 0.0335, 149, 0.35, 0.48]
['0610B', 0.0005, 143, 0.3, 0.48]
['0632A', 0.0724, 55, 0.28, 0.47]
['0635A', 0.0991, 73, 0.34, 0.49]
['0672A', 0.143, 82, 0.34, 0.48]
['0720A', 0.106, 77, 0.33, 0.48]
['0739A', 0.1427, 73, 0.34, 0.48]
['0819A', 0.0809, 90, 0.35, 0.49]
['0820A', 0.0349, 41, 0.23, 0.47]
['0869A', 0.0, 207, 0.04, 0.47]
['0871B', 0.2278, 83, 0.35, 0.47]
['0873A', 0.0737, 55, 0.29, 0.47]
['0878A', 0.2308, 67, 0.34, 0.47]
['0911A', 0.0352, 127, 0.33, 0.47]
['0911B', 0.0684, 120, 0.35, 0.48]
['0915A', 0.1584, 83, 0.34, 0.47]
['0919B', 0.2645,

['Ottawa_CONT66', 0.0, 118, 0.07, 0.47]
['Ottawa_CONT790', 0.0, 123, 0.06, 0.47]
['Ottawa_CONT816', 0.0, 79, 0.12, 0.47]
['Ottawa_CONT869', 0.0, 68, 0.1, 0.47]
['Ottawa_CONT908', 0.0007, 60, 0.2, 0.47]
['ESP_2047', 0.0, 101, 0.07, 0.5]
['ESP_2056', 0.006, 45, 0.21, 0.49]
['ESP_2057', 0.0062, 66, 0.26, 0.49]
['ESP_2060', 0.0, 66, 0.1, 0.5]
['ESP_2062', 0.0, 57, 0.12, 0.5]
['ESP_2837', 0.0, 178, 0.07, 0.51]
['ESP_2855', 0.0, 123, 0.09, 0.49]
['ESP_3206', 0.0002, 150, 0.31, 0.5]
['ESP_3218', 0.0013, 85, 0.27, 0.5]
['ESP_3220', 0.0262, 69, 0.31, 0.5]
['ESP_3222', 0.0638, 96, 0.35, 0.5]
['ESP_3287', 0.0323, 93, 0.33, 0.5]
['ESP_3302', 0.0856, 55, 0.31, 0.5]
['ESP_3781', 0.0, 80, 0.11, 0.53]
['ESP_3813', 0.0, 77, 0.09, 0.5]
['ESP_3952', 0.0, 80, 0.2, 0.5]
['ESP_3958', 0.0, 117, 0.06, 0.5]
['ESP_4003', 0.0, 65, 0.14, 0.5]
['ESP_4093', 0.0, 149, 0.07, 0.49]
['ESP_4388', 0.0, 92, 0.08, 0.5]
['ESP_4402', 0.0, 77, 0.2, 0.5]
['ESP_4404', 0.0, 100, 0.1, 0.5]
['ESP_4406', 0.0004, 103, 0.27, 0.5]
['E

['PROMIS191036347', 0.22, 72, 0.34, 0.47]
['PROMIS191036375', 0.0515, 90, 0.33, 0.48]
['PROMIS191036443', 0.1211, 74, 0.34, 0.48]
['PROMIS191036447', 0.1343, 80, 0.34, 0.48]
['PROMIS191036472', 0.1449, 62, 0.33, 0.48]
['PROMIS191036482', 0.1036, 82, 0.35, 0.49]
['PROMIS191036600', 0.1548, 86, 0.35, 0.48]
['PROMIS191036604', 0.1639, 70, 0.34, 0.48]
['PROMIS191036606', 0.1164, 77, 0.35, 0.49]
['PROMIS191036704', 0.035, 101, 0.33, 0.48]
['PROMIS191036764', 0.0804, 77, 0.33, 0.49]
['PROMIS191036771', 0.0334, 83, 0.31, 0.48]
['PROMIS191036816', 0.064, 100, 0.34, 0.48]
['PROMIS191036832', 0.0306, 48, 0.25, 0.48]
['PROMIS191036868', 0.1937, 66, 0.34, 0.47]
['PROMIS191036876', 0.1862, 70, 0.35, 0.49]
['PROMIS191036918', 0.0205, 76, 0.33, 0.52]
['PROMIS191036919', 0.0, 93, 0.09, 0.49]
['PROMIS191036951', 0.1551, 59, 0.33, 0.48]
['PROMIS191036952', 0.0486, 90, 0.32, 0.48]
['PROMIS191036973', 0.128, 63, 0.34, 0.49]
['PROMIS191036980', 0.2253, 60, 0.35, 0.49]
['PROMIS191036983', 0.054, 71, 0.31, 0

...

['10608', 0.0001, 110, 0.27, 0.5]
['11113', 0.0, 166, 0.04, 0.47]
['11330', 0.0, 144, 0.09, 0.5]
['11649', 0.0346, 139, 0.33, 0.46]
['11802', 0.0, 110, 0.09, 0.5]
['1241', 0.1026, 82, 0.35, 0.49]
['12682', 0.0, 76, 0.17, 0.49]
['12735', 0.0, 81, 0.21, 0.48]
['13147', 0.0, 85, 0.13, 0.49]
['13160', 0.0, 85, 0.14, 0.5]
['14273', 0.0, 112, 0.08, 0.49]
['14653', 0.0264, 119, 0.33, 0.48]
['15017', 0.0, 152, 0.28, 0.54]
['15554', 0.0001, 84, 0.22, 0.48]
['15935', 0.0, 182, 0.29, 0.49]
['16108', 0.0681, 109, 0.35, 0.49]
['16488', 0.0, 167, 0.05, 0.49]
['17259', 0.001, 72, 0.24, 0.49]
['17288', 0.0, 106, 0.15, 0.48]
['18600', 0.0, 99, 0.18, 0.48]
['18990', 0.0295, 50, 0.27, 0.48]
['21208', 0.0038, 93, 0.29, 0.5]
['21368', 0.0917, 87, 0.34, 0.48]
['21491', 0.1563, 47, 0.32, 0.49]
['21536', 0.0, 113, 0.17, 0.52]
['21556', 0.0, 84, 0.11, 0.48]
['21619', 0.0, 127, 0.09, 0.5]
['21957', 0.003, 86, 0.27, 0.48]
['22262', 0.0, 96, 0.21, 0.49]
['22614', 0.0, 94, 0.21, 0.52]
['23046', 0.0, 82, 0.13, 0.48

['PT-1TBB', 0.039, 145, 0.34, 0.47]
['PT-1TBL', 0.265, 66, 0.35, 0.47]
['PT-1TCJ', 0.1644, 117, 0.35, 0.46]
['PT-2862', 0.0, 72, 0.1, 0.54]
['PT-286W', 0.0577, 117, 0.34, 0.47]
['PT-28HN', 0.0, 154, 0.06, 0.53]
['PT-295G', 0.0, 141, 0.05, 0.48]
['PT-296H', 0.0034, 52, 0.22, 0.48]
['PT-29BR', 0.0, 120, 0.06, 0.45]
['PT-2M1H', 0.0012, 60, 0.21, 0.49]
['PT-2M2H', 0.2009, 51, 0.35, 0.5]
['PT-2M6D', 0.0623, 105, 0.35, 0.49]
['PT-3MGB', 0.0498, 53, 0.28, 0.48]
['PT-3MNR', 0.0, 73, 0.16, 0.48]
['PT-7X55', 0.0028, 122, 0.3, 0.48]
['PT-7YW5', 0.0, 175, 0.33, 0.53]
['PT-7Z6W', 0.0718, 55, 0.31, 0.49]
['PT-8TE6', 0.0, 138, 0.07, 0.49]
['PT-8TEZ', 0.0, 102, 0.1, 0.47]
['PT-8TJ5', 0.1606, 101, 0.34, 0.45]
['PT-8TJX', 0.0, 113, 0.1, 0.49]
['PT-8TV8', 0.0, 162, 0.06, 0.48]
['PT-8TWM', 0.0, 151, 0.05, 0.45]
['PT-8UCQ', 0.0, 89, 0.08, 0.46]
['PT-8VHI', 0.0, 143, 0.05, 0.46]
['PT-8VY9', 0.0, 181, 0.11, 0.5]
['PT-8W44', 0.0, 124, 0.06, 0.47]
['PT-8WEI', 0.0974, 75, 0.33, 0.48]
['PT-8WF5', 0.0, 118, 0.08,

['SM-1PDQB', 0.0, 107, 0.1, 0.5]
['SM-1PDS9', 0.0844, 92, 0.35, 0.49]
['SM-1PDSA', 0.1199, 68, 0.34, 0.49]
['SM-1PDSF', 0.0047, 74, 0.27, 0.49]
['SM-1PDSY', 0.1297, 85, 0.35, 0.48]
['SM-1PDTJ', 0.0, 120, 0.08, 0.55]
['SM-1PDUC', 0.0115, 143, 0.34, 0.49]
['SM-1PDV8', 0.0818, 108, 0.35, 0.48]
['SM-1PDVB', 0.0921, 56, 0.32, 0.49]
['SM-1PDVK', 0.0, 150, 0.08, 0.5]
['SM-1PDX2', 0.0854, 92, 0.35, 0.49]
['SM-1PDXE', 0.006, 140, 0.32, 0.48]
['SM-1PDYO', 0.0, 174, 0.05, 0.5]
['SM-1PE19', 0.0508, 122, 0.35, 0.48]
['SM-1PE1R', 0.2232, 91, 0.34, 0.45]
['SM-1PE1T', 0.0276, 127, 0.35, 0.49]
['SM-1PE23', 0.159, 75, 0.35, 0.49]
['SM-1PE24', 0.0456, 120, 0.35, 0.49]
['SM-1PHPQ', 0.1697, 81, 0.35, 0.48]
['SM-1PHQ3', 0.0001, 65, 0.16, 0.46]
['SM-1PHQ7', 0.1268, 73, 0.33, 0.47]
['SM-1PHTD', 0.0827, 88, 0.35, 0.49]
['SM-1PHTX', 0.0384, 54, 0.27, 0.48]
['SM-1PHU7', 0.0546, 92, 0.33, 0.48]
['SM-1PHUG', 0.1391, 65, 0.34, 0.49]
['SM-1PI1Y', 0.1959, 123, 0.35, 0.45]
['SM-1PI31', 0.3221, 75, 0.35, 0.46]
['SM-1PI

['SM-2IYNA', 0.2862, 76, 0.35, 0.46]
['SM-2IYNS', 0.0, 130, 0.12, 0.51]
['SM-2IYO5', 0.0, 108, 0.06, 0.46]
['SM-2IYOF', 0.0, 81, 0.15, 0.46]
['SM-2IYP2', 0.0, 96, 0.07, 0.49]
['SM-2IYP5', 0.0, 146, 0.06, 0.52]
['SM-2IYPB', 0.0, 170, 0.05, 0.53]
['SM-2IYPS', 0.2001, 104, 0.34, 0.45]
['SM-2IYR8', 0.0, 123, 0.06, 0.5]
['SM-2IYRV', 0.0, 135, 0.06, 0.45]
['SM-2IYSO', 0.0, 89, 0.19, 0.46]
['SM-2IYSY', 0.0, 113, 0.06, 0.5]
['SM-2IYT7', 0.0423, 230, 0.35, 0.45]
['SM-2IYTE', 0.3403, 65, 0.34, 0.45]
['SM-2IYTG', 0.1543, 113, 0.34, 0.45]
['SM-2IYTS', 0.0, 125, 0.06, 0.53]
['SM-2IYTT', 0.0, 102, 0.17, 0.47]
['SM-2IYUJ', 0.1552, 99, 0.35, 0.47]
['SM-2IYUY', 0.15728427157284272, 31, 0.21, 0.46]
['SM-2J43S', 0.0, 133, 0.05, 0.47]
['SM-2J44U', 0.0, 127, 0.06, 0.52]
['SM-2J4U6', 0.0, 85, 0.08, 0.5]
['SM-2K69J', 0.0001, 63, 0.1, 0.46]
['SM-2K6AX', 0.1201, 101, 0.35, 0.48]
['SM-2K6BT', 0.0, 131, 0.14, 0.48]
['SM-2K6CC', 0.2128, 81, 0.35, 0.47]
['SM-2K6CS', 0.0, 111, 0.07, 0.47]
['SM-2K6IE', 0.0774, 106, 

<Figure size 432x288 with 0 Axes>

In [2]:
### if depth comes from picard metrics ###

plt.gcf().clear()
iterations=10000

in_file="/Users/mleventh/Documents/COPD/all_clone/noLod/reannotated/top_clone_5mut.txt"
maf=pd.read_csv(in_file,sep='\t')

maf=maf.dropna(subset=['germline_cluster'])
maf=maf.reset_index(drop=True)

maf_subset=maf.loc[:,["Tumor_Sample_Barcode","median_coverage","CCF_hat_01","germline_cluster"]]

print(len(maf_subset["Tumor_Sample_Barcode"].unique()))

maf_subset=maf_subset.drop_duplicates(subset=["Tumor_Sample_Barcode"])
maf_subset['median_coverage']=maf_subset['median_coverage'].astype(int)
maf_subset=maf_subset.reset_index(drop=True)

confusion_df=pd.DataFrame(columns=["Tumor_Sample_Barcode","FDR"])

for idx,row in maf_subset.iterrows():

    somatic_sim_alt=stats.binom.rvs(row['median_coverage'],row['CCF_hat_01'],size=iterations)
    germ_sim_alt=stats.binom.rvs(row['median_coverage'],row['germline_cluster'],size=iterations)
    
    sim_germ_germ=stats.beta.pdf(row['germline_cluster'],germ_sim_alt+1,row['median_coverage']-germ_sim_alt+1)
    sim_germ_somatic=stats.beta.pdf(row['germline_cluster'],somatic_sim_alt+1,row['median_coverage']-somatic_sim_alt+1)
    
    sim_somatic_germ=stats.beta.pdf(row['CCF_hat_01'],germ_sim_alt+1,row['median_coverage']-germ_sim_alt+1)
    sim_somatic_somatic=stats.beta.pdf(row['CCF_hat_01'],somatic_sim_alt+1,row['median_coverage']-somatic_sim_alt+1)
    
    som_lod=np.subtract(np.log10(sim_somatic_somatic),np.log10(sim_germ_somatic))
    germ_lod=np.subtract(np.log10(sim_somatic_germ),np.log10(sim_germ_germ))
    
    min_lik=min(min(som_lod),min(germ_lod))
    max_lik=max(max(som_lod),max(germ_lod))
    #print([min_lik,max_lik])
#     min_lik=-10
#     max_lik=10
    
    threshold=np.arange(min_lik,max_lik,abs(min_lik-max_lik)/99)
    
    true_positive=[]
    false_negative=[]
    true_negative=[]
    false_positive=[]

    for theta in threshold:

        true_positive.append(len(som_lod[som_lod>=theta]))
        false_negative.append(len(som_lod[(som_lod<theta)|(somatic_sim_alt<=3)]))
        true_negative.append(len(germ_lod[(germ_lod<theta)|(germ_sim_alt<=3)]))
        false_positive.append(len(germ_lod[germ_lod>=theta]))

    tp=np.array(true_positive)/(np.array(true_positive)+np.array(false_negative))
    fp=np.array(false_positive)/(np.array(false_positive)+np.array(true_negative))
    fd=np.array(false_positive)/(np.array(false_positive)+(1/1000)*np.array(true_positive))
    

    if len(np.where(tp>=0.9)[0])>0:
        print([row['Tumor_Sample_Barcode'],min(fd[tp>=0.9]),row['median_coverage'],row['CCF_hat_01'],
              row['germline_cluster']])
        
        df=pd.DataFrame([[row['Tumor_Sample_Barcode'],min(fd[tp>=0.9])]],
                                 index=[0],columns=["Tumor_Sample_Barcode","FDR"])
        
        confusion_df=confusion_df.append(df,ignore_index=True)
    else:
        df=pd.DataFrame([[row['Tumor_Sample_Barcode'],0]],
                       index=[0],columns=["Tumor_Sample_Barcode","FDR"])
        confusion_df=confusion_df.append(df,ignore_index=True)


#confusion_df.to_csv('confusion.txt',sep='\t',index=False)

confusion_df=confusion_df.loc[confusion_df['FDR']<0.1,:]

maf=maf[maf["Tumor_Sample_Barcode"].isin(confusion_df["Tumor_Sample_Barcode"])]

maf.to_csv("/Users/mleventh/Documents/COPD/all_clone/noLod/reannotated/top_clone_5mut_depthLim.txt",sep='\t',index=False)
    
#plt.savefig("fdr_by_purity_and_depth.png")
    
    #print(confusion_df)
    

878
['5804_5804', 0.9984890588844394, 80, 0.48, 0.54]
['5805_5805', 0.0, 94, 0.2, 0.49]
['5808_5808', 0.244299674267101, 97, 0.25, 0.49]
['5810_5810', 0.9952781912675736, 104, 0.38, 0.49]
['5814_5814', 0.9940943289377882, 112, 0.39, 0.5]


/Users/mleventh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in true_divide


['5818_5818', 0.17530020159523182, 92, 0.24, 0.49]
['5824_5824', 0.5624824224242992, 65, 0.22, 0.49]
['5829_5829', 0.9980895230086682, 94, 0.46, 0.53]
['5832_5832', 0.0, 85, 0.19, 0.5]
['C2GN3ACXX-5-ID10', 0.6099544567338973, 44, 0.16, 0.49]
['COPDGene_A41943_6343', 0.9981914744577608, 91, 0.43, 0.5]
['COPDGene_A53453_3853', 0.0, 57, 0.12, 0.49]
['COPDGene_A95641_3859', 0.9982009191952007, 61, 0.44, 0.53]
['COPDGene_B71593_3865', 0.9950576998267843, 75, 0.37, 0.5]
['COPDGene_B96157_3868', 0.0, 67, 0.15, 0.49]
['COPDGene_C03191_3870', 0.9982779013594311, 76, 0.42, 0.5]
['COPDGene_C16158_6405', 0.0, 87, 0.21, 0.5]
['COPDGene_C19536_3872', 0.9956974611560334, 73, 0.36, 0.49]
['COPDGene_C77510_3880', 0.0, 71, 0.1, 0.5]
['COPDGene_D15395_7254', 0.9926047711310442, 107, 0.38, 0.5]
['COPDGene_D16246_7240', 0.9984945329044506, 43, 0.41, 0.5]
['COPDGene_D37448_3888', 0.9986361897696159, 63, 0.46, 0.52]
['COPDGene_D46279_3889', 0.9982871301337007, 74, 0.42, 0.5]
['COPDGene_D97472_3897', 0.997190

['S-000447293_97155', 0.3422079255355554, 50, 0.14, 0.47]
['S-000447330_97746', 0.8230130114438, 51, 0.21, 0.47]
['S-000447331_97440', 0.9832278013740869, 32, 0.22, 0.5]
['S-000447334_97738', 0.16972165648336726, 55, 0.13, 0.47]
['S-000447352_97418', 0.9944336694305024, 54, 0.32, 0.48]
['S-000447353_97135', 0.17838030681412773, 56, 0.17, 0.48]
['S-000447356_96897', 0.9972754995527486, 51, 0.36, 0.49]
['S-000447357_97565', 0.8904230058921818, 46, 0.14, 0.47]
['S-000447360_97465', 0.24809791597750577, 49, 0.2, 0.49]
['S-000447365_97753', 0.1762580417731559, 59, 0.17, 0.48]
['S-000447366_97286', 0.9958156371190624, 54, 0.35, 0.49]
['S-000447379_97176', 0.5307855626326964, 49, 0.15, 0.48]
['S-000447417_97740', 0.0, 62, 0.17, 0.48]
['S-000447438_97028', 0.6660123387549075, 43, 0.17, 0.48]
['S-000447465_97707', 0.6318865763595435, 49, 0.19, 0.48]
['S-000447491_97072', 0.9645054862922772, 40, 0.16, 0.49]
['S-000447524_97103', 0.9911274195740384, 50, 0.3, 0.48]
['S-000447528_96936', 0.98293809

['S-000631591_39439', 0.9988148796968707, 40, 0.43, 0.48]
['S-000633335_98011', 0.9972758194942404, 52, 0.37, 0.5]
['S-000634361_98008', 0.9972830711272027, 56, 0.38, 0.5]
['S-000634404_98022', 0.9975618161034824, 65, 0.42, 0.52]
['S-000637559_98025', 0.997890580857264, 54, 0.39, 0.5]
['S-000646996_97890', 0.9977592195655176, 45, 0.42, 0.54]
['S-000653618_98014', 0.998128408535005, 83, 0.42, 0.5]
['S-000666812_39648', 0.997596818218908, 48, 0.37, 0.49]
['S-000666828_97974', 0.9976844214797571, 124, 0.43, 0.5]
['S-000666988_39473', 0.9984589583011657, 55, 0.44, 0.52]
['S-000667083_39393', 0.9978662246782023, 93, 0.42, 0.5]
['S-000667102_97977', 0.9972186047811801, 47, 0.36, 0.49]
['S-000672549_97887', 0.996944919853064, 41, 0.34, 0.49]
['S-000947733_98073', 0.998054911630424, 71, 0.4, 0.49]
['S-000970788_98068', 0.9979905822630308, 120, 0.43, 0.5]
['S-000981479_98069', 0.9976794021524014, 48, 0.42, 0.55]
['S-000984045_98071', 0.0, 58, 0.14, 0.48]
['S-000984058_97893', 0.9977862879118393

<Figure size 432x288 with 0 Axes>

Estimating fdr rate by clone

In [23]:
# purity_range=np.arange(0.01,0.51,by=0.01)
# depth_range=np.arange(10,410,by=10)

plt.gcf().clear()
iterations=10000

in_file="all_clone/SVM_predicted_phylogic_muts_filt.txt"
maf=pd.read_csv(in_file,sep='\t')

maf_subset=maf.loc[:,["Tumor_Sample_Barcode","t_alt_count","t_ref_count","CCF_hat_01","Cluster_ID","germline_cluster"]]

print(len(maf_subset["Tumor_Sample_Barcode"].unique()))

maf_subset['depth']=maf_subset.t_ref_count+maf_subset.t_alt_count

median_depths=maf_subset.groupby(['Tumor_Sample_Barcode'])['depth'].median().reset_index(name='median_depth')

maf_subset=pd.merge(maf_subset,median_depths,on="Tumor_Sample_Barcode")
maf_subset=maf_subset.drop_duplicates(subset=["Tumor_Sample_Barcode","Cluster_ID"])
maf_subset['median_depth']=maf_subset['median_depth'].astype(int)
maf_subset=maf_subset.reset_index(drop=True)

confusion_df=pd.DataFrame(columns=["Tumor_Sample_Barcode","Cluster_ID","FDR"])

for idx,row in maf_subset.iterrows():

    somatic_sim_alt=stats.binom.rvs(row['median_depth'],row['CCF_hat_01'],size=iterations)
    germ_sim_alt=stats.binom.rvs(row['median_depth'],row['germline_cluster'],size=iterations)
    
    sim_germ_germ=stats.beta.pdf(row['germline_cluster'],germ_sim_alt+1,row['median_depth']-germ_sim_alt+1)
    sim_germ_somatic=stats.beta.pdf(row['germline_cluster'],somatic_sim_alt+1,row['median_depth']-somatic_sim_alt+1)
    
    sim_somatic_germ=stats.beta.pdf(row['CCF_hat_01'],germ_sim_alt+1,row['median_depth']-germ_sim_alt+1)
    sim_somatic_somatic=stats.beta.pdf(row['CCF_hat_01'],somatic_sim_alt+1,row['median_depth']-somatic_sim_alt+1)
    
    som_lod=np.subtract(np.log10(sim_somatic_somatic),np.log10(sim_germ_somatic))
    germ_lod=np.subtract(np.log10(sim_somatic_germ),np.log10(sim_germ_germ))
    
    min_lik=min(min(som_lod),min(germ_lod))
    max_lik=max(max(som_lod),max(germ_lod))
    #print([min_lik,max_lik])
#     min_lik=-10
#     max_lik=10
    
    threshold=np.arange(min_lik,max_lik,abs(min_lik-max_lik)/99)
    
    true_positive=[]
    false_negative=[]
    true_negative=[]
    false_positive=[]

    for theta in threshold:

        true_positive.append(len(som_lod[som_lod>=theta]))
        false_negative.append(len(som_lod[(som_lod<theta)|(somatic_sim_alt<=3)]))
        true_negative.append(len(germ_lod[(germ_lod<theta)|(germ_sim_alt<=3)]))
        false_positive.append(len(germ_lod[germ_lod>=theta]))

    tp=np.array(true_positive)/(np.array(true_positive)+np.array(false_negative))
    fp=np.array(false_positive)/(np.array(false_positive)+np.array(true_negative))
    fd=np.array(false_positive)/(np.array(false_positive)+(1/1000)*np.array(true_positive))
    
#     plt.plot(fd,tp,label=row["Tumor_Sample_Barcode"])
#     plt.xlabel("FDR")
#     plt.ylabel("TPR")
#     plt.legend(loc='left')
    

    if len(np.where(tp>=0.9)[0])>0:
        print([row['Tumor_Sample_Barcode'],row['Cluster_ID'],min(fp[tp>=0.9]),row['median_depth'],row['CCF_hat_01'],
              row['germline_cluster']])
        
        df=pd.DataFrame([[row['Tumor_Sample_Barcode'],row["Cluster_ID"],min(fd[tp>=0.9])]],
                                 index=[0],columns=["Tumor_Sample_Barcode","Cluster_ID","FDR"])
        
        confusion_df=confusion_df.append(df,ignore_index=True)
    else:
        df=pd.DataFrame([[row['Tumor_Sample_Barcode'],row["Cluster_ID"],1]],
                       index=[0],columns=["Tumor_Sample_Barcode","Cluster_ID","FDR"])
        confusion_df=confusion_df.append(df,ignore_index=True)


#confusion_df.to_csv('confusion.txt',sep='\t',index=False)

confusion_df=confusion_df.loc[confusion_df['FDR']<0.1,:]

maf=maf[(maf["Tumor_Sample_Barcode"].isin(confusion_df["Tumor_Sample_Barcode"]))&
       maf["Cluster_ID"].isin(confusion_df["Cluster_ID"])]

maf.to_csv("all_clone/depth_lim_SVM_pred.txt",sep='\t',index=False)
    
#plt.savefig("fdr_by_purity_and_depth.png")
    
    #print(confusion_df)
    

8918
['SM-2J4P7', 2, 0.0084, 111, 0.28, 0.45]
['PROMIS191039693', 3, 0.0957, 91, 0.34, 0.48]
['PROMIS191038337', 3, 0.0918, 82, 0.34, 0.48]
['SM-1PDJ3', 3, 0.0849, 81, 0.34, 0.49]
['SM-1PD5T', 2, 0.022, 134, 0.34, 0.48]
['0919B', 3, 0.2695, 66, 0.35, 0.47]
['SM-2J3YY', 3, 0.2452, 102, 0.35, 0.45]
['SM-2J3YY', 4, 0.0, 102, 0.07, 0.45]
['SM-22BU6', 3, 0.0852, 87, 0.33, 0.47]
['SM-22BPW', 2, 0.0526, 74, 0.31, 0.48]
['SM-22BMQ', 4, 0.0, 130, 0.24, 0.47]
['0539A', 3, 0.0, 95, 0.08, 0.47]
['0539A', 2, 0.1378, 95, 0.35, 0.47]
['SM-271JR', 3, 0.0, 98, 0.07, 0.52]
['SM-2K6SX', 4, 0.0001, 62, 0.18, 0.48]
['SM-2K6SX', 3, 0.2782, 62, 0.35, 0.48]
['SM-1JBPV', 5, 0.0, 81, 0.11, 0.5]
['SM-2IYSM', 4, 0.0, 110, 0.06, 0.53]
['SM-2IYSM', 3, 0.0, 110, 0.2, 0.53]
['0930A', 4, 0.0, 130, 0.05, 0.46]
['SM-2J4RP', 4, 0.0, 109, 0.06, 0.49]
['SM-2J43L', 4, 0.0, 94, 0.07, 0.48]
['SM-2IYQT', 5, 0.0, 148, 0.05, 0.47]
['SM-2IYQT', 4, 0.0, 148, 0.16, 0.47]
['SM-1PI56', 4, 0.0, 113, 0.07, 0.46]
['Taiwanese_Trios_10C11

/Users/mleventh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in true_divide


['PROMIS191038547', 3, 0.0, 83, 0.16, 0.47]
['PROMIS191038590', 4, 0.0, 69, 0.15, 0.54]
['PROMIS191038590', 3, 0.0, 69, 0.23, 0.54]
['PROMIS191038590', 2, 0.0168, 69, 0.33, 0.54]
['SM-2J4BR', 4, 0.0, 95, 0.07, 0.45]
['SM-2J4BR', 3, 0.1336, 95, 0.33, 0.45]
['SM-2J4HW', 4, 0.0, 104, 0.08, 0.5]
['Taiwanese_Trios_10C104446', 3, 0.0, 95, 0.07, 0.54]
['Taiwanese_Trios_10C104839', 4, 0.0, 115, 0.06, 0.46]
['Taiwanese_Trios_10C105050', 4, 0.0, 128, 0.06, 0.55]
['Taiwanese_Trios_10C105050', 3, 0.0, 128, 0.27, 0.55]
['PT-2862', 3, 0.0, 72, 0.1, 0.54]
['CC_GB035427', 7, 0.0, 129, 0.08, 0.52]
['CC_GB035427', 4, 0.0, 129, 0.27, 0.52]
['CC_GB035427', 6, 0.0, 129, 0.17, 0.52]
['SM-271IS', 3, 0.0, 112, 0.08, 0.49]
['SM-2IYKD', 5, 0.0, 114, 0.06, 0.52]
['SM-2IYKD', 4, 0.0, 114, 0.16, 0.52]
['SM-2IYKD', 3, 0.0, 114, 0.26, 0.52]
['SM-1PIJ8', 6, 0.0, 114, 0.07, 0.53]
['SM-1PIJ8', 5, 0.0, 114, 0.17, 0.53]
['SM-22TXU', 3, 0.007, 70, 0.26, 0.48]
['SM-2B79H', 5, 0.0, 122, 0.07, 0.52]
['SM-2B79H', 3, 0.0, 122,

['CC_GB031897', 12, 0.0, 141, 0.05, 0.5]
['CC_GB031897', 11, 0.0, 141, 0.1, 0.5]
['CC_GB031897', 8, 0.0, 141, 0.26, 0.5]
['CC_GB031897', 10, 0.0, 141, 0.19, 0.5]
['CC_GB031897', 9, 0.0, 141, 0.19, 0.5]
['0111B', 2, 0.1961, 50, 0.32, 0.47]
['SM-2B79C', 5, 0.0, 95, 0.07, 0.49]
['SM-2B79C', 4, 0.0, 95, 0.14, 0.49]
['SM-2B7G1', 4, 0.0, 109, 0.07, 0.53]
['SM-2B7G1', 3, 0.0, 109, 0.15, 0.53]
['SM-1PI4U', 5, 0.0, 113, 0.07, 0.48]
['SM-2J46K', 4, 0.0, 100, 0.07, 0.53]
['SM-2B78M', 4, 0.0, 138, 0.06, 0.5]
['SM-1PI85', 2, 0.1897, 102, 0.34, 0.45]
['SM-1PI85', 3, 0.0, 102, 0.07, 0.45]
['SM-2J4TS', 4, 0.0, 118, 0.06, 0.54]
['SM-1PI7X', 5, 0.0, 113, 0.07, 0.46]
['SM-1PIA6', 6, 0.0, 92, 0.08, 0.5]
['SM-1PIA6', 5, 0.0, 92, 0.2, 0.5]
['SM-1PIHF', 3, 0.0, 113, 0.07, 0.45]
['SM-2J45E', 4, 0.0, 92, 0.08, 0.52]
['SM-1JBNY', 5, 0.0, 129, 0.1, 0.5]
['SM-1JBNY', 4, 0.001, 129, 0.31, 0.5]
['15935', 3, 0.0035, 95, 0.29, 0.49]
['SM-2IYLH', 5, 0.0, 131, 0.06, 0.5]
['SM-2IYLH', 4, 0.0, 131, 0.18, 0.5]
['SM-1UJF1'

['SM-2J3Y4', 4, 0.0, 113, 0.07, 0.5]
['SM-2J42Z', 6, 0.0001, 128, 0.3, 0.52]
['SM-2J4BQ', 4, 0.0, 97, 0.08, 0.5]
['1404B', 4, 0.0, 81, 0.09, 0.48]
['1404B', 3, 0.0324, 81, 0.3, 0.48]
['SM-2IYRR', 3, 0.0, 108, 0.07, 0.5]
['SM-2J4UX', 4, 0.0, 95, 0.07, 0.51]
['SM-2J497', 3, 0.0, 114, 0.2, 0.45]
['SM-2J3ZI', 6, 0.0, 119, 0.07, 0.46]
['SM-2J3ZI', 5, 0.0, 119, 0.2, 0.46]
['SM-2J483', 3, 0.2562, 88, 0.34, 0.45]
['SM-2J494', 4, 0.0, 87, 0.08, 0.51]
['Taiwanese_Trios_09C91162', 4, 0.0, 99, 0.07, 0.46]
['Taiwanese_Trios_09C91162', 3, 0.2098, 99, 0.35, 0.46]
['SM-1PI8P', 4, 0.0, 112, 0.07, 0.49]
['SM-1JCZB', 3, 0.0, 82, 0.12, 0.48]
['SM-1JCZB', 2, 0.1472, 82, 0.35, 0.48]
['SM-2J4FC', 4, 0.0, 119, 0.06, 0.46]
['ESP_3824', 3, 0.0, 57, 0.13, 0.5]
['SM-271C2', 5, 0.0, 111, 0.08, 0.52]
['SM-271C2', 4, 0.0, 111, 0.26, 0.52]
['ESP_4160', 4, 0.0, 68, 0.12, 0.55]
['ESP_4160', 3, 0.0, 68, 0.23, 0.55]
['SM-2J49U', 4, 0.0, 101, 0.07, 0.51]
['SM-2B78A', 4, 0.0, 99, 0.08, 0.54]
['SM-2J4GT', 4, 0.0, 109, 0.06,

...

['TCGA_TCGA-C4-A0F7-10A-01D-A10S-08', 4, 0.0, 166, 0.07, 0.48]
['340754-48', 2, 0.0, 107, 0.06, 0.48]
['SM-1R3HX', 3, 0.0, 82, 0.1, 0.54]
['365125-59', 2, 0.0023, 167, 0.33, 0.49]
['id34166', 3, 0.0415, 114, 0.35, 0.49]
['SM-1R2R7', 2, 0.0, 70, 0.19, 0.47]
['PT-1SCE', 2, 0.0, 120, 0.09, 0.47]
['pcath981452-1', 3, 0.0545, 92, 0.34, 0.49]
['SM-2K6KK', 3, 0.0184, 95, 0.29, 0.46]
['A07121_realigned', 3, 0.0403, 87, 0.33, 0.49]
['8825912', 3, 0.0, 99, 0.14, 0.49]
['PT-8WKG', 3, 0.0047, 130, 0.31, 0.48]
['PT-8WKG', 4, 0.0, 130, 0.05, 0.48]
['SM-22TO1', 2, 0.0, 141, 0.1, 0.48]
['SM-473', 2, 0.0471, 84, 0.33, 0.49]
['REGICOR_A0500003843007132801', 2, 0.0, 94, 0.16, 0.48]
['SM-1EN3U', 3, 0.0, 103, 0.22, 0.46]
['SM-1EN3U', 4, 0.0, 103, 0.07, 0.46]
['Ottawa_CONT1474', 2, 0.0089, 76, 0.26, 0.47]
['Ottawa_CONT1474', 3, 0.0, 76, 0.12, 0.47]
['SM-1PDF7', 4, 0.0024, 114, 0.31, 0.5]
['SM-1R31U', 4, 0.018, 89, 0.35, 0.53]
['id52689', 3, 0.0289, 75, 0.31, 0.49]
['PROMIS191035908', 3, 0.0421, 100, 0.33, 0

['Taiwanese_Trios_10C117022', 2, 0.0, 62, 0.11, 0.47]
['SM-1JBKW', 4, 0.0, 83, 0.09, 0.47]
['SM-1QY9M', 4, 0.0, 86, 0.09, 0.49]
['SM-1QY9M', 3, 0.1131, 86, 0.35, 0.49]
['PROMIS191031325', 3, 0.0122, 104, 0.31, 0.48]
['SM-2K6I9', 3, 0.0, 99, 0.14, 0.46]
['Ottawa_CONT1174', 3, 0.0026, 61, 0.22, 0.47]
['PROMIS191037787', 3, 0.1267, 67, 0.33, 0.48]
['38201', 6, 0.0, 297, 0.04, 0.5]
['38201', 4, 0.0, 297, 0.35, 0.5]
['38201', 5, 0.0, 297, 0.1, 0.5]
['SM-1JBUB', 3, 0.0138, 130, 0.32, 0.48]
['SM-1R3C8', 3, 0.0, 93, 0.07, 0.46]
['SM-26UJ9', 4, 0.0, 119, 0.14, 0.49]
['PT-ISO7', 4, 0.0, 217, 0.06, 0.48]
['PROMIS191032832', 3, 0.1302, 69, 0.33, 0.48]
['A14112_realigned', 3, 0.0, 108, 0.23, 0.49]
['A14112_realigned', 2, 0.0231, 108, 0.33, 0.49]
['SM-22Z7Y', 3, 0.0, 134, 0.07, 0.48]
['SM-22Z7Y', 2, 0.0, 134, 0.22, 0.48]
['SM-1JCS9', 4, 0.0436, 100, 0.34, 0.49]
['SM-1JCS9', 5, 0.0, 100, 0.09, 0.49]
['Ottawa_P508', 3, 0.0, 74, 0.09, 0.47]
['PT-OPQQ', 4, 0.0, 100, 0.09, 0.49]
['Ottawa_CONT2329', 3, 0.

['A07764_realigned', 3, 0.0035, 99, 0.29, 0.49]
['PROMIS191034958', 3, 0.0355, 94, 0.32, 0.48]
['391853-81', 2, 0.0105, 171, 0.35, 0.49]
['Ottawa_CONT883', 3, 0.0342, 80, 0.3, 0.47]
['TCGA_TCGA-06-0210-10A-01D-1491-08', 2, 0.0205, 102, 0.31, 0.48]
['TCGA_TCGA-06-0210-10A-01D-1491-08', 4, 0.0, 102, 0.13, 0.48]
['SM-22C9S', 3, 0.0, 310, 0.33, 0.48]
['SM-22C9S', 4, 0.0, 310, 0.05, 0.48]
['SM-2K68I', 3, 0.0305, 92, 0.31, 0.47]
['Ottawa_CONT2365', 2, 0.1216, 94, 0.35, 0.48]
['Ottawa_CHD193', 3, 0.0, 104, 0.07, 0.47]
['REGICOR_A1100008281007072801', 3, 0.0059, 53, 0.23, 0.48]
['PROMIS191032287', 3, 0.0145, 96, 0.31, 0.49]
['PROMIS191032287', 4, 0.0, 96, 0.07, 0.49]
['PROMIS191037859', 3, 0.0918, 69, 0.32, 0.48]
['PT-BPK1', 2, 0.0, 124, 0.21, 0.47]
['SM-1FLQQ', 3, 0.0, 78, 0.09, 0.47]
['ESP_3373', 4, 0.0184, 82, 0.31, 0.5]
['ESP_3373', 5, 0.0, 82, 0.14, 0.5]
['SM-22CAT', 3, 0.0096, 91, 0.28, 0.47]
['SM-22CAT', 2, 0.0263, 91, 0.3, 0.47]
['SM-1EMRX', 3, 0.0, 131, 0.06, 0.47]
['TCGA_TCGA-77-A5GH

['PROMIS191031487', 4, 0.0005, 147, 0.3, 0.49]
['PROMIS191031487', 5, 0.0, 147, 0.05, 0.49]
['Taiwanese_Trios_10C107634', 3, 0.0284, 48, 0.26, 0.48]
['PT-FG1H', 3, 0.2338, 59, 0.34, 0.48]
['PT-FG1H', 4, 0.0, 59, 0.13, 0.48]
['1222A', 5, 0.0, 199, 0.04, 0.49]
['1222A', 4, 0.0, 199, 0.13, 0.49]
['id35606', 3, 0.0011, 71, 0.24, 0.49]
['PT-L1YJ', 2, 0.0246, 112, 0.32, 0.47]
['PT-1RVD', 2, 0.0506, 150, 0.35, 0.47]
['PT-1RVD', 3, 0.0, 150, 0.16, 0.47]
['SM-1PDWQ', 4, 0.014, 142, 0.34, 0.49]
['SM-1PDWQ', 5, 0.0, 142, 0.11, 0.49]
['PROMIS191033955', 3, 0.1843, 56, 0.33, 0.48]
['id52443', 3, 0.0249, 47, 0.26, 0.49]
['PROMIS191031375', 3, 0.1165, 59, 0.31, 0.48]
['PROMIS191038100', 4, 0.0, 65, 0.13, 0.52]


<Figure size 432x288 with 0 Axes>

Merging top clone with next clone over

In [9]:
def second_largest(numbers):
    count = 0
    m1 = m2 = float('-inf')
    for x in numbers:
        count += 1
        if x > m2:
            if x >= m1:
                m1, m2 = x, m1            
            else:
                m2 = x
    return m2 if count >= 2 else 0

In [32]:
top_clone="indel_muts/short_indel_SNV.maf"
all_clone="all_clone/SVM_predicted_phylogic_muts_filt_noMutLim.txt"

top_maf=pd.read_csv(top_clone,sep='\t')
top_maf['clonal_peak']=top_maf['CCF_hat_01']
all_maf=pd.read_csv(all_clone,sep='\t')

all_maf=all_maf[all_maf['Tumor_Sample_Barcode'].isin(top_maf['Tumor_Sample_Barcode'])]
all_maf=all_maf.reset_index(drop=True)

top_maf=top_maf.append(all_maf,sort=False)

print(len(top_maf.index))
top_maf=top_maf.drop_duplicates(subset=['Tumor_Sample_Barcode','Genome_Change'],keep='first')
print(len(top_maf.index))
top_maf=top_maf.reset_index(drop=True)

#top_maf=top_maf[top_maf['clonal_peak']<=top_maf['CCF_hat_01']]
print(len(top_maf.index))

filtered_count=top_maf.groupby(['Tumor_Sample_Barcode','CCF_hat_01']).size().reset_index(name='counts')

count_chip=filtered_count[filtered_count.counts>=4]

merge_maf=top_maf.merge(count_chip,on=['Tumor_Sample_Barcode','CCF_hat_01'])

merge_maf['subclonal_CCF']=merge_maf.groupby(['Tumor_Sample_Barcode']).apply(lambda x: second_largest(x['CCF_hat_01']))

top_maf.to_csv("indel_muts/subclone_add/short_indel_SNV_subclone.txt",sep='\t',index=False)


22851
12730
12730


(1) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $LOD_{T}(m,f)\geq \log_{10}(\delta_{T})-\log_{10}(\frac{P(m,f)}{1-P(m,f)})$ &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\delta_{T}=80$

(2) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\frac{\beta(alt+1,ref+1,\frac{purity*0.4}{2*\theta_{germline}})}{\beta(alt+1,ref+1,0.4)+\beta(alt+1,ref+1,\frac{purity*0.4}{2*\theta_{germline}})} > 0.9 \implies PASS$

(3) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $alt_{somatic}=B(depth,\frac{purity}{2})$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $alt_{germline}=B(depth,\theta_{germline})$

(4) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $LOD_{S}=\log_{10}(\frac{\beta(alt_{somatic}+1,depth-alt_{somatic}+1,\frac{purity}{2})}{\beta(alt_{somatic}+1,depth-alt_{somatic}+1,\theta_{germline})})$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $LOD_{G}=\log_{10}(\frac{\beta(alt_{germline}+1,depth-alt_{germline}+1,\frac{purity}{2})}{\beta(alt_{germline}+1,depth-alt_{germline}+1,\theta_{germline})})$

(5) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\theta \in [min(LOD_{S},LOD_{G}),\dots,min(LOD_{S},LOD_{G})+N*\frac{max(LOD_{S},LOD{G})-min(LOD_{S},LOD_{G})}{100},\dots,max(LOD_{S},LOD{G})]$ &nbsp;&nbsp;&nbsp; $N\in[1,100]$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $LOD_{S}>\theta \implies TP$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $LOD_{S}<\theta$ &nbsp; or &nbsp; $alt_{somatic}<3 \implies FP$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $LOD_{G}<\theta$ &nbsp; or &nbsp; $alt_{germline}<3 \implies TN$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $LOD_{G}>\theta \implies FN$ <br>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $TPR=\frac{TP}{TP+FN}$ <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $FDR=\frac{FP}{FP+10^{-3}*TP}$

(6) &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; $\frac{\beta(alt+1,ref+1,\frac{purity}{2})}{\beta(alt+1,ref+1,\theta_{predicted_germline})+\beta(alt+1,ref+1,\frac{purity}{2})} > 0.9 \implies PASS$

In [37]:
### identify limits on number of mutations per clone ###

input_file="all_clone/all_clone_mut_lim/all_clone_noLim_age.txt"
maf=pd.read_csv(input_file,sep='\t')

maf=maf[maf['mut_per_clone']>1]
maf.reset_index(drop=True)

maf['lambda']=(.12*maf['age'])
print(maf[['lambda','mut_per_clone','age']])

maf=maf[maf['mut_per_clone']>=maf['lambda']]
print(len(maf['Tumor_Sample_Barcode'].unique()))

maf.to_csv("all_clone/all_clone_mut_lim/all_clone_ageMutLeyLim.txt",sep='\t',index=False)

       lambda  mut_per_clone   age
0        5.16              2  43.0
1        5.16              2  43.0
2        5.04              2  42.0
3        5.04              2  42.0
5        4.80              3  40.0
6        4.80              3  40.0
7        4.80              3  40.0
10       4.56              2  38.0
11       4.56              2  38.0
13       4.68              2  39.0
14       4.68              2  39.0
18       4.80              2  40.0
20       4.80              2  40.0
22       4.92              4  41.0
23       4.92              4  41.0
24       4.92              4  41.0
25       4.92              4  41.0
28       4.92              3  41.0
29       4.92              3  41.0
30       4.92              3  41.0
31       3.60              3  30.0
32       3.60              3  30.0
33       3.60              3  30.0
34       4.20              2  35.0
35       4.20              2  35.0
36       4.20              2  35.0
37       4.20              2  35.0
38       5.04       